#### Arcpy + Geopandas combination

In [1]:
#Must be opened from ArcGIS Python Command Prompt (juptyter lab)
# Must have signed into ArcGIS online? (run ArcGIS Pro)

import arcpy
from arcpy import env
from arcpy.sa import *

import pandas as pd
import geopandas as gpd

from pathlib import Path
from datetime import datetime

In [2]:
def renameShapeField(inshp,infieldname,outfieldname):
    '''renames a field in a shapefile using geopandas because arcpy doesn't support this.'''
    ingdf = gpd.read_file(inshp)
    ingdf.rename(columns={infieldname: outfieldname}, inplace=True)
    ingdf.to_file(inshp)
    print(f'Fieldname {infieldname} changed to {outfieldname} in shapefile.')

In [52]:
#inputs
#DEMS
# dem1 = Path(r"D:\Whiskeytown\dem_diff\brandy_creek\dems\orig\brandy_dem2019.tif")
# dem2 = Path(r"D:\Whiskeytown\dem_diff\brandy_creek\dems\orig\brandy_dem2020.tif")
dem1 = Path(r"D:\Whiskeytown\dem_diff\brandy_creek\dems\orig\brandy_dem2018.tif")
dem2 = Path(r"D:\Whiskeytown\dem_diff\brandy_creek\dems\orig\brandy_dem2019.tif")

#Stable polygons
# stablepolyshp = Path(r"D:\Whiskeytown\dem_diff\brandy_creek\shp\Stable_poly_19-20_expanded.shp")
stablepolyshp = Path(r"D:\Whiskeytown\dem_diff\brandy_creek\shp\Stable_poly_18-19_expanded.shp")

#Output dod name stem
dodnamestem = r"DoD_19-18"
detrendnamestem = r"Detrend_19-18_polyn"

#output dir
demdiff_dir = Path(r"D:\Whiskeytown\dem_diff\brandy_creek\demdiff19-18")
outdod_dir = Path(demdiff_dir, r"dod")
outdod_unadj_dir  = Path(demdiff_dir, r"dod\unadj")
outdod_adj_dir  = Path(demdiff_dir, r"dod\adj")
outdod_pt_dir  = Path(demdiff_dir, r"dod\shp")
outtrendraster_dir = Path(demdiff_dir, r"detrend")
outtrend_pt_dir = Path(demdiff_dir, r"detrend\shp")
scratch_dir = Path(demdiff_dir, r"arcpyscratch")

#create parent if doesn't exist
demdiff_dir.mkdir(parents=True, exist_ok=True)
#create subdir if don't exist
for direc in [outdod_dir, outdod_unadj_dir, outdod_adj_dir, outdod_pt_dir, outtrendraster_dir, outtrend_pt_dir, scratch_dir]:
    direc.mkdir(parents=True, exist_ok=True)

# # Set local variables
# instablepointfeatures = Path(r"D:\Whiskeytown\dem_diff\brandy_creek\dod\shp\DoD_20-19_unadj_pts_stable.shp")

In [50]:
# Set environment settings
arcpy.env.workspace = str(scratch_dir)
arcpy.env.overwriteOutput = True
arcpy.env.compression = "LZW"

env.extent = str(dem1)
env.snapRaster = str(dem1)

# Check out the ArcGIS Spatial Analyst extension license
arcpy.CheckOutExtension("Spatial");

In [51]:
#Make unadjusted DoD
outunadjdod = Path(outdod_unadj_dir,dodnamestem + r'_unadj.tif')

dod = RasterCalculator([str(dem1), str(dem2)], ["dem1", "dem2"],
                                       "dem2-dem1", "FirstOf", "FirstOf")

dod.save(str(outunadjdod))


In [136]:
#Convert DoD to point shapefile, then clip with stable polygons and rename field to 'dod_unadj'
#inputs
outstablept_shp = str(Path(outdod_pt_dir, outunadjdod.stem + r'_stable_pts.shp')) #same as DoD, but '_stable_pts.shp')

#covert to points scratchfile
arcpy.RasterToPoint_conversion(dod, r"memory\tempRasPt", "VALUE");
#clip scratchfile
arcpy.Clip_analysis(r"memory\tempRasPt", str(stablepolyshp), r"memory\tempRasPtClip", "");
#get polygon "type (paved, unpaved)" onto att table via spatial join
arcpy.analysis.SpatialJoin(r"memory\tempRasPtClip", str(stablepolyshp), outstablept_shp);

#rename field
renameShapeField(outstablept_shp,'grid_code','dod_unadj')


Fieldname grid_code changed to dod_unadj in shapefile.


In [75]:
#Calculate trend surface using all stable points (polynomial 0, 1 and 2)

#Inputs/outputs
stablept_shp = outstablept_shp #(from cell above)
outtrendrasterstem = str(Path(outtrendraster_dir, detrendnamestem))

zField = "dod_unadj"
cellSize = 0.25
# PolynomialOrder = 2 (set in loop)
regressionType = "LINEAR"

for polyn_order in [0, 1, 2]:
    # Execute Trend
    print(f'Creating trend surface polynomial order {polyn_order}...')
    outTrend = Trend(stablept_shp, zField, cellSize, 
                     polyn_order, regressionType)
    outTrend.save(outtrendrasterstem + str(polyn_order) + ".tif")


Creating trend surface polynomial order 0...
Creating trend surface polynomial order 1...
Creating trend surface polynomial order 2...


In [97]:
# '''
# Apply to DoD to evaluate visually
# Adjustment order is:
#     DoD - TrendRaster = AdjustedDoD
# This is equivalent to adjusting DEM1:
#      DEM2 - DEM1 - TrendRaster = AdjustedDoD
# Since TrendRaster = DoD residual which should be zero.
# For example:
# DEM2 = 3, DEM1 = 5, DoD = -2, TrendRaster = -2
# AdjustedDoD =  DEM2 - DEM1 - TrendRaster = 0
#             =  3 - 5 -(-2) = 0
#             =  DoD - TrendRaster = 0
#             =  -2 -(-2) = 0
# '''

#inputs/outputs
inunadjdod = str(outunadjdod) #from above cell
intrendrasterstem = outtrendrasterstem #from cell above
outadjdodstem = str(Path(outdod_adj_dir, dodnamestem + r'_adj_polyn_'))

#loop through detrend surfaces and apply to DoD
for polyn_order in [0,1,2]:
    #apply adjustment
    trendraster = intrendrasterstem + str(polyn_order) + '.tif'
    adj = RasterCalculator([inunadjdod, trendraster], ["dod", "trendraster"],
                                       "dod-trendraster", "FirstOf", "FirstOf")
    adj.save(outadjdodstem + str(polyn_order) + '.tif')
    

In [162]:
#Extract residual values from each adjusted DoD trend surface and evaluate bulk points

stablept_shp = str(Path(outdod_pt_dir, outunadjdod.stem + r'_stable_pts.shp')) #same as DoD, but '_pts.shp')
inadjdodstem = str(Path(outdod_adj_dir, dodnamestem + r'_adj_polyn_'))
outdetrendevalptshp = str(Path(outtrend_pt_dir, Path(outdod_pt_dir, outunadjdod.stem + r'_pts.shp') .stem + r'_detrend_eval.shp'))

#copy stable points to eval file
arcpy.Copy_management(stablept_shp, outdetrendevalptshp);

#derive list of adjusted dods and output fieldnames for multi-raster extract
inras_outfield_list = [[inadjdodstem + str(n) + '.tif', 'poly' + str(n) + 'resid'] for n in [0,1,2]]

#extract adjusted dod residual values to points
ExtractMultiValuesToPoints(outdetrendevalptshp, inras_outfield_list, "NONE")

#read into gdf to evaluate
tempgdf = gpd.read_file(outdetrendevalptshp)

#evaluate paved/unpaved
pavedgdfstat = tempgdf[tempgdf['type'] == 'paved'].drop(columns=['geometry','pointid', 'TARGET_FID','Join_Count','id']).describe()
pavedgdfstat['type'] = 'paved'
pavedgdfstat['stat'] = pavedgdfstat.index
unpavedgdfstat = tempgdf[tempgdf['type'] == 'unpaved'].drop(columns=['geometry','pointid', 'TARGET_FID','Join_Count','id']).describe()
unpavedgdfstat['type'] = 'unpaved'
unpavedgdfstat['stat'] = unpavedgdfstat.index
tempgdfstat = tempgdf.drop(columns=['geometry','pointid', 'TARGET_FID','Join_Count','id']).describe()
tempgdfstat['type'] = 'all'
tempgdfstat['stat'] = tempgdfstat.index

dfstat = pd.concat([tempgdfstat, pavedgdfstat, unpavedgdfstat], ignore_index=True)

# Evaluate bulk residual values and write to file
dfstat.to_csv(Path(outtrendraster_dir, detrendnamestem + r'bulk_pt_adjustment_eval.csv'))
print(dfstat)


        dod_unadj    poly0resid    poly1resid     poly2resid     type   stat
0   336555.000000  3.365550e+05  3.365550e+05  336555.000000      all  count
1       -0.183162  3.892094e-07  3.535217e-07      -0.000075      all   mean
2        0.212388  2.123879e-01  1.908409e-01       0.181654      all    std
3       -6.121185 -5.938023e+00 -5.933698e+00      -5.915244      all    min
4       -0.275787 -9.262459e-02 -9.024733e-02      -0.084777      all    25%
5       -0.149628  3.353508e-02  1.134694e-03       0.007044      all    50%
6       -0.059570  1.235925e-01  9.501526e-02       0.090722      all    75%
7        0.970703  1.153866e+00  1.291805e+00       1.191392      all    max
8   264783.000000  2.647830e+05  2.647830e+05  264783.000000    paved  count
9       -0.166343  1.681957e-02  4.446654e-03      -0.013171    paved   mean
10       0.221408  2.214079e-01  2.009906e-01       0.188935    paved    std
11      -6.121185 -5.938023e+00 -5.933698e+00      -5.915244    paved    min

### Decide which trend surface to use

In [215]:
# Subdivide stable polygons to use as test/train sets for residual error evaluation, then get poly and subdiv poly id onto points via intersection

#inputs
# target_subpoly_area = 9 # (9 = 144 points)size of subpolygons in m^2 (too small may result in overfitting of trend surface to validation points?)
target_subpoly_area = 100 # (100 = 1600 pts) size of subpolygons in m^2(too small may result in overfitting of trend surface to validation points?)
#points
stablept_shp = str(Path(outdod_pt_dir, outunadjdod.stem + r'_stable_pts.shp')) #same as DoD, but '_pts.shp')
outpointswithsubpolyid_str =  str(Path(outdod_pt_dir, outunadjdod.stem + r'_stable_pts' + str(target_subpoly_area) + r'm_subpolyid.shp'))

#polygons

stablepolyshp_str = str(stablepolyshp)
outstablepolysubdivided_str = str(Path(str(stablepolyshp.with_suffix('')) + '_' + str(target_subpoly_area) + 'm.shp'))

#subdivide polygon
print('Subdividing polygons ...')
arcpy.SubdividePolygon_management(
    stablepolyshp_str, outstablepolysubdivided_str, "EQUAL_AREAS","", target_subpoly_area, "", "", 
    "STACKED_BLOCKS");

#add subdiv poly id to poly attr table
polygdf = gpd.read_file(outstablepolysubdivided_str)
polygdf['subpolyid'] = polygdf.index
polygdf.to_file(outstablepolysubdivided_str)

#spatial join stable points with subdiv polygons to get polyid and subdividedpolyid on points for later test/train filter.
print('Joining sub-polygon id to stable points ...')
ptgdf = gpd.read_file(stablept_shp)

ptwithpolygdf = gpd.sjoin(ptgdf, polygdf, how="left", op='intersects')
ptwithpolygdf.drop(columns=['id_right', 'type_right','index_right'], inplace=True)
ptwithpolygdf.rename(columns = {'id_left': 'id', 'type_left': 'type'}, inplace=True)

#write output shp
ptwithpolygdf.to_file(outpointswithsubpolyid_str)

print(f'Subpolygons written to: \n     {outstablepolysubdivided_str}')
print(f'Stable points with sub-polygon id written to: \n     {outpointswithsubpolyid_str}')

In [ ]:
#Evaluate Residual Error

#order of polynomial for trend surface
PolynomialOrder = 1

#Iterate and do test/train splits to evaluate by subpolygon
print(datetime.now().strftime("%Y/%d/%m %H:%M:%S"))    
#number of iterations (~2-3 minutes each?)
num_loops = 1

#inputs
pointswithsubpolyid = outpointswithsubpolyid_str #from cell above
stablepolysubdivided = outstablepolysubdivided_str #from cell above

#load points with subpoly id into gdf to be filtered
ptwithpolygdf = gpd.read_file(pointswithsubpolyid)

#trend surface details
zField = "dod_unadj"
cellSize = 0.25
regressionType = "LINEAR"

#result df
resultsdf_subpoly = pd.DataFrame(columns = ['bulk_pt_mean','bulk_pt_std','all_poly_mean','all_poly_std',
                                            'paved_poly_mean','paved_poly_std','unpaved_poly_mean','unpaved_poly_std'])
resultsdf_poly = pd.DataFrame(columns = ['bulk_pt_mean','bulk_pt_std','all_poly_mean','all_poly_std',
                                         'paved_poly_mean','paved_poly_std','unpaved_poly_mean','unpaved_poly_std'])
#percent of poly to use for train 
trainfrac = 0.6

for i in range(num_loops):
    #clean up scratch dir first (leave results of final iteration in scratch to let user see them)
    for fn in [r"\temp_trainpoly.shp", r"\temp_valpoly.shp", r"\temp_trainpoint.shp", r"\temp_valpoint.shp", 
               r"\temp_valpoint_withdetrend.shp",r"\temp_trendraster.tif"]:
        arcpy.management.Delete(arcpy.env.workspace + fn)
    print(f'Beginning iteration {i} ...')
    #subset poly to test/train
    arcpy.ga.SubsetFeatures(stablepolysubdivided, 
                            arcpy.env.workspace + r"\temp_trainpoly.shp", 
                            arcpy.env.workspace + r"\temp_valpoly.shp", 60, "PERCENTAGE_OF_INPUT")
    #load each to get poly id
    trainpolygdf = gpd.read_file(arcpy.env.workspace + r"\temp_trainpoly.shp")
    valpolygdf = gpd.read_file(arcpy.env.workspace + r"\temp_valpoly.shp")
    
    #use subpolyid to select train/val points
    trainptgdf = ptwithpolygdf[ptwithpolygdf['subpolyid'].isin(trainpolygdf['subpolyid'].tolist())]
    valptgdf = ptwithpolygdf[ptwithpolygdf['subpolyid'].isin(valpolygdf['subpolyid'].tolist())]
    
    #write to temp
    trainptgdf.to_file(arcpy.env.workspace + r"\temp_trainpoint.shp")
    valptgdf.to_file(arcpy.env.workspace + r"\temp_valpoint.shp")
    
    #create trend surface
    # Execute Trend
    print('Creating trend surface...')
    outTrend = Trend(arcpy.env.workspace + r"\temp_trainpoint.shp", zField, cellSize, 
                     PolynomialOrder, regressionType)
    outTrend.save(arcpy.env.workspace + r"\temp_trendraster.tif")
    
    #Sample raster on validate points
    print('Sampling trend surface...')
    ExtractValuesToPoints(arcpy.env.workspace + r"\temp_valpoint.shp", 
                          arcpy.env.workspace + r"\temp_trendraster.tif", 
                          arcpy.env.workspace + r"\temp_valpoint_withdetrend.shp",
                          "NONE", "VALUE_ONLY")
    
    #read into gdf to eval
    print('Evaluating trend surface...')
    evalgdf = gpd.read_file(arcpy.env.workspace + r"\temp_valpoint_withdetrend.shp")
    #apply to DoD value (dod_unadj - trendRasterValue)
    evalgdf['resid'] = evalgdf['dod_unadj'] - evalgdf['RASTERVALU']
    
    #gather residuals grouped by subpolygons
    data_subpoly = {'bulk_pt_mean': [evalgdf['resid'].mean()],
            'bulk_pt_std': [evalgdf['resid'].std()],
            'all_poly_mean': [evalgdf.groupby('subpolyid')['resid'].mean().mean()],
            'all_poly_std': [evalgdf.groupby('subpolyid')['resid'].mean().std()],
            'paved_poly_mean': [evalgdf[evalgdf['type'] == 'paved'].groupby('subpolyid')['resid'].mean().mean()],
            'paved_poly_std': [evalgdf[evalgdf['type'] == 'paved'].groupby('subpolyid')['resid'].mean().std()],
            'unpaved_poly_mean': [evalgdf[evalgdf['type'] == 'unpaved'].groupby('subpolyid')['resid'].mean().mean()],
            'unpaved_poly_std': [evalgdf[evalgdf['type'] == 'unpaved'].groupby('subpolyid')['resid'].mean().std()]
           }
    resultsdf_subpoly = resultsdf_subpoly.append(pd.DataFrame(data, 
                                              columns = ['bulk_pt_mean','bulk_pt_std','all_poly_mean','all_poly_std',
                                                         'paved_poly_mean','paved_poly_std','unpaved_poly_mean','unpaved_poly_std']), 
                                 ignore_index=True)
    
    #gather residuals grouped by entire polygons
    data_poly = {'bulk_pt_mean': [evalgdf['resid'].mean()],
            'bulk_pt_std': [evalgdf['resid'].std()],
            'all_poly_mean': [evalgdf.groupby('id')['resid'].mean().mean()],
            'all_poly_std': [evalgdf.groupby('id')['resid'].mean().std()],
            'paved_poly_mean': [evalgdf[evalgdf['type'] == 'paved'].groupby('id')['resid'].mean().mean()],
            'paved_poly_std': [evalgdf[evalgdf['type'] == 'paved'].groupby('id')['resid'].mean().std()],
            'unpaved_poly_mean': [evalgdf[evalgdf['type'] == 'unpaved'].groupby('id')['resid'].mean().mean()],
            'unpaved_poly_std': [evalgdf[evalgdf['type'] == 'unpaved'].groupby('id')['resid'].mean().std()]
           }
    resultsdf_poly = resultsdf_subpoly.append(pd.DataFrame(data, 
                                              columns = ['bulk_pt_mean','bulk_pt_std','all_poly_mean','all_poly_std',
                                                         'paved_poly_mean','paved_poly_std','unpaved_poly_mean','unpaved_poly_std']), 
                                 ignore_index=True)
    
    
    print(datetime.now().strftime("%Y/%d/%m %H:%M:%S"))   
        
    #write intermediate output, overwrite to save progress
    resultsdf_subpoly.to_csv(Path(outtrendraster_dir, detrendnamestem + r'_order_' + str(PolynomialOrder) + r'_adjustment_eval_by_' + str(target_subpoly_area) + r'm_polygon.csv'))
    resultsdf_subpoly.describe().to_csv(Path(outtrendraster_dir, detrendnamestem + r'_order_' + str(PolynomialOrder) + 
                                     r'_adjustment_eval_by_' + str(target_subpoly_area) + r'm_polygon_summary_stats.csv'))
    
    resultsdf_poly.to_csv(Path(outtrendraster_dir, detrendnamestem + r'_order_' + str(PolynomialOrder) + r'_adjustment_eval_by_whole_polygon.csv'))
    resultsdf_poly.describe().to_csv(Path(outtrendraster_dir, detrendnamestem + r'_order_' + str(PolynomialOrder) + 
                                     r'_adjustment_eval_by_whole_polygon_summary_stats.csv'))



2021/29/01 17:50:32
Beginning iteration 0 ...


### Same as above, but using entire polygons instead of subidived polygons

In [36]:
#iterate and do test/train splits to evaluate by whole polygon
print(datetime.now().strftime("%Y/%d/%m %H:%M:%S"))    
#number of iterations (~2-3 minutes each?)
num_loops = 10
#inputs
pointswithsubpolyid = r"D:\Whiskeytown\dem_diff\brandy_creek\dod\shp\DoD_20-19_unadj_pts_stable_subpolyid.shp"
# stablepolysubdivided = r"D:\Whiskeytown\dem_diff\brandy_creek\shp\Stable_poly_19-20_expanded_subdivided.shp"
stablepolysubdivided = r"D:\Whiskeytown\dem_diff\brandy_creek\shp\Stable_poly_19-20_expanded.shp"

#load points with subpoly id into gdf to be filtered
ptwithpolygdf = gpd.read_file(pointswithsubpolyid)

#trend surface details
zField = "dod_unadj"
cellSize = 0.25
PolynomialOrder = 2
regressionType = "LINEAR"

#result df
resultsdf = pd.DataFrame(columns = ['bulk_pt_mean','bulk_pt_std','all_poly_mean','all_poly_std','paved_poly_mean','paved_poly_std','unpaved_poly_mean','unpaved_poly_std'])
#percent of poly to use for train 
trainfrac = 0.6

for i in range(num_loops):
    #clean up scratch dir first (leave results of final iteration in scratch to let user see them)
    for fn in [r"\temp_trainpoly.shp", r"\temp_valpoly.shp", r"\temp_trainpoint.shp", r"\temp_valpoint.shp", r"\temp_valpoint_withdetrend.shp",r"\temp_trendraster.tif"]:
        arcpy.management.Delete(arcpy.env.workspace + fn)
    print(f'Beginning iteration {i} ...')
    #subset poly to test/train
    arcpy.ga.SubsetFeatures(stablepolysubdivided, 
                            arcpy.env.workspace + r"\temp_trainpoly.shp", 
                            arcpy.env.workspace + r"\temp_valpoly.shp", 60, "PERCENTAGE_OF_INPUT")
    #load each to get poly id
    trainpolygdf = gpd.read_file(arcpy.env.workspace + r"\temp_trainpoly.shp")
    valpolygdf = gpd.read_file(arcpy.env.workspace + r"\temp_valpoly.shp")
    
    #use subpolyid to select train/val points
    trainptgdf = ptwithpolygdf[ptwithpolygdf['id'].isin(trainpolygdf['id'].tolist())]
    valptgdf = ptwithpolygdf[ptwithpolygdf['id'].isin(valpolygdf['id'].tolist())]
    
    #write to temp
    trainptgdf.to_file(arcpy.env.workspace + r"\temp_trainpoint.shp")
    valptgdf.to_file(arcpy.env.workspace + r"\temp_valpoint.shp")
    
    #create trend surface
    # Execute Trend
    print('Creating trend surface...')
    outTrend = Trend(arcpy.env.workspace + r"\temp_trainpoint.shp", zField, cellSize, 
                     PolynomialOrder, regressionType)
    outTrend.save(arcpy.env.workspace + r"\temp_trendraster.tif")
    
    #Sample raster on validate points
    print('Sampling trend surface...')
    ExtractValuesToPoints(arcpy.env.workspace + r"\temp_valpoint.shp", 
                          arcpy.env.workspace + r"\temp_trendraster.tif", 
                          arcpy.env.workspace + r"\temp_valpoint_withdetrend.shp",
                          "NONE", "VALUE_ONLY")
    
    #read into gdf to eval
    print('Evaluating trend surface...')
    evalgdf = gpd.read_file(arcpy.env.workspace + r"\temp_valpoint_withdetrend.shp")
    #apply to DoD value (dod_unadj - trendRasterValue)
    evalgdf['resid2'] = evalgdf['dod_unadj'] - evalgdf['RASTERVALU']
    
    #gather residuals grouped by polygons
    data = {'bulk_pt_mean': [evalgdf['resid2'].mean()],
            'bulk_pt_std': [evalgdf['resid2'].std()],
            'all_poly_mean': [evalgdf.groupby('id')['resid2'].mean().mean()],
            'all_poly_std': [evalgdf.groupby('id')['resid2'].mean().std()],
            'paved_poly_mean': [evalgdf[evalgdf['type'] == 'paved'].groupby('id')['resid2'].mean().mean()],
            'paved_poly_std': [evalgdf[evalgdf['type'] == 'paved'].groupby('id')['resid2'].mean().std()],
            'unpaved_poly_mean': [evalgdf[evalgdf['type'] == 'unpaved'].groupby('id')['resid2'].mean().mean()],
            'unpaved_poly_std': [evalgdf[evalgdf['type'] == 'unpaved'].groupby('id')['resid2'].mean().std()]
           }
    resultsdf = resultsdf.append(pd.DataFrame(data, 
                                              columns = ['bulk_pt_mean','bulk_pt_std','all_poly_mean','all_poly_std','paved_poly_mean','paved_poly_std','unpaved_poly_mean','unpaved_poly_std']), 
                                 ignore_index=True)
    
    
    print(datetime.now().strftime("%Y/%d/%m %H:%M:%S"))  
    
    #write intermediate output, overwrite to save progress
    resultsdf.to_csv(r"D:\Whiskeytown\dem_diff\brandy_creek\detrend_surface\ResidualSystematicErrorByWholePolygon_20-19.csv")
    resultsdf.describe().to_csv(r"D:\Whiskeytown\dem_diff\brandy_creek\detrend_surface\ResidualSystematicErrorByWholePolygon_SummaryStats_20-19.csv")



2021/28/01 13:29:39
Beginning iteration 0 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 13:31:19
Beginning iteration 1 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 13:34:18
Beginning iteration 2 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 13:35:43
Beginning iteration 3 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 13:37:11
Beginning iteration 4 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 13:39:40
Beginning iteration 5 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 13:42:17
Beginning iteration 6 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend surface...
2021/28/01 13:45:16
Beginning iteration 7 ...
Creating trend surface...
Sampling trend surface...
Evaluating trend sur